In [1]:
from datetime import datetime

In [2]:
# Variables almost remains constant
debt_rate = 0.074 # percent

In [3]:
class Parameters:
    def __init__(self, **kwds):
        self.__dict__.update(kwds)

In [4]:
# Corpus with interface: deposit() and a withdraw() function 
class Corpus:
    def __init__(self):
        self._balance = 0
        self._date = datetime(1990, 1, 1)

    def Check(self, date):
        if (date < self._date):
            print("Date cannot be less than last account operation date")
            return False
        return True

In [5]:
# DebtCorpus with both a deposit() and a withdraw() function 
class DebtCorpus(Corpus):
  
    def Deposit(self, date, amount):
        if (not self.Check(date)):
            return False
        delta = (date - self._date).days / 365.25
        self._balance = amount + self._balance * pow(1 + debt_rate, delta)
        self._date = date
        return True
    
    def Get(self, date):
        delta = (date - self._date).days / 365.25
        self._balance = self._balance * pow(1 + debt_rate, delta)
        self._date = date
        return self._balance 
  
    def Withdraw(self, date, amount): 
        if (not self.Check(date)):
            return False
        self.Get(date)
        fulfilled = min(self._balance, amount)
        if self._balance < amount:
            print('Balance is lesser than requested amount', self._balance, amount)
        self._balance -= fulfilled 
        return fulfilled

In [6]:
# EquityCorpus with both a deposit() and a withdraw() function 
class EquityCorpus(Corpus):
    def __init__(self, benchmark):
        self._benchmark = benchmark
        self._date = datetime(1990, 1, 1)
        self._balance = 0
    
    def _NextDate(self, date):
        return self._benchmark.index[self._benchmark.index >= date][0].to_pydatetime()
    
    def _Get(self, next_date):
        returns = self._benchmark.loc[next_date]['Close'] / self._benchmark.loc[self._date]['Close']
        self._balance = self._balance * returns
        return self._balance
  
    def Deposit(self, date, amount):
        next_date = self._NextDate(date)
        if not self.Check(next_date):
            return False
        if 0 == self._balance:
            self._balance = amount
        else:
            self._Get(next_date)
            self._balance += amount
        self._date = next_date
        return True
    
    def Get(self, date):
        next_date = self._NextDate(date)
        r = self._Get(next_date)
        self._date = next_date
        return r
    
    def Withdraw(self, date, amount):
        next_date = self._NextDate(date)
        if not self.Check(next_date):
            return False
        self._Get(next_date)
        self._date = next_date
        fulfilled = min(self._balance, amount)
        if self._balance < amount:
            print('Balance is lesser than requested', self._balance, amount)
        self._balance -= fulfilled
        return fulfilled